# Introduction

This Jupyter notebook example demonstrates how to run a time domain response reconstruction analysis in Sesam, using Sesam Core. This example is based on the EMULF Delta Floater model.

OneWorkflow is used to orchestrate the analysis. 

The analysis in this notebook contains the following steps:
1. Run pre-compute in Sesam Core. This is done once.
2. Read design load case information from an Excel spreadsheet
3. For each design load case, run Response Reconstruction and fatigue analysis in Sesam Core
4. FLS result accumulation

<img src="_img/EMULF_Deltafloater.png" alt="image" width="30%" height="auto">

## 1. Input variables

Some variables are defined up front. By default, they are set to values corresponding to the example model


In [1]:
from pathlib import Path
import sys
import os

root_folder = Path(os.getcwd())

### USER INPUT ###

# Use these variables to control which parts of the pipeline you want to run
run_precompute = True
run_reconstruction = True
run_accumulation = True

# ID of this OneWorkflow workspace
workspace_id = "ResponseReconstructionFLS"  # workspace id

# True if you want to run on cloud, False if you want to run locally (Note: Cloud support is coming soon!)
cloud_run = False

# Path to the workspace folder, containing all the analysis input and output files
workspace_path = root_folder / "Workspace"

# Prefix to the FEM file
fem_file_prefix = ""

# Super element number of the top super element
top_sup = 3

### Precompute part
# Folder where the precompute will be run
precom_execution_folder = r"Precompute\Execution"

# Folder where the precompute output will be stored
precom_output_folder = r"Precompute\Output"

# Path to the CommonFiles folder. In this example, all input files are found here
common_files_folder = "CommonFiles"

# Name of Excel file containing DLC information
excel_file = "parameter_input.xlsx"

### END USER INPUT ###


# Add the PythonModules folder to the system path so that we can import modules from there
sys.path.append(str(Path(root_folder, "../PythonModules")))



## 2. Pre-computation


Pre-computation in Sesam Core is run once

Declare required folders and variables for pre-computation, generate an input file, and execute Sesam Core pre-computation

In [2]:
import shutil
from dnv.sesam.commands import SesamCoreCommand
from dnv.sesam.commands.executors import execute_command
from dnv.oneworkflow import CreateInputFileFromFileTemplateCommand, CommandInfo


# Define the paths to the folders and files
execution_folder_path = workspace_path / precom_execution_folder
output_folder_path = workspace_path / precom_output_folder
input_folder_path = workspace_path / common_files_folder
precompute_input_file = execution_folder_path / "precompute_input.json"
precompute_input_file_template = "precompute_input_template.json"

# Deleting existing analysis folder, unless user doesn't want to run pre-compute
if run_precompute is True:
    shutil.rmtree(execution_folder_path, ignore_errors=True)
    shutil.rmtree(output_folder_path, ignore_errors=True)

# Ensure that the folders exist
execution_folder_path.mkdir(parents=True, exist_ok=True)
output_folder_path.mkdir(parents=True, exist_ok=True)

# Note the use of Path.as_posix(): This will write the paths with forward slashes ('/') into the input file
# The alternative is to make sure that the paths contain double back-slashes ('\\') in the json file
precompute_parameters = {
    "inputpath": str(input_folder_path.as_posix()),
    "outputpath": str(output_folder_path.as_posix()),
    "executepath": str(execution_folder_path.as_posix()),
    "simaforce": str((input_folder_path / "sima.force").as_posix()),
    "femfile": str((input_folder_path / f"{fem_file_prefix}T{top_sup}.FEM").as_posix()),
}

if run_precompute:
    # Create a command to generate the precompute_input.json by taking the precompute_input_template.json and replacing the strings with parameters
    precompute_input_command = CreateInputFileFromFileTemplateCommand(
        template_input_file = precompute_input_file_template,
        input_filename = str(precompute_input_file),
        parameters = precompute_parameters,
        working_dir = str(input_folder_path),
    )

    print("Generating input files for precompute")
    execute_command(precompute_input_command)
    print("Input files for precompute generated")

    score_precompute = SesamCoreCommand(
        working_dir = str(execution_folder_path)
    )

    score_precompute.arguments = (
        "precompute unit-response-json -i " + str(precompute_input_file)
    )
    print("Running precompute command")
    execute_command(score_precompute)
    print("Precompute command finished")


INFO: The .NET Runtime Path 'C:\Users\kblu\AppData\Roaming\Python\Python312\site-packages\dnv\net\runtime\.net'
INFO: The .NET Runtime Version '.NET 8.0.4'. Runtime Identifier 'win-x64'.

Generating input files for precompute
Input files for precompute generated
Running precompute command
Precompute command finished


#### Verifying that the pre-computation was successful

In [3]:
import analysisStatusChecker

print("Verifying pre-compute command result:")
logFileToCheck = output_folder_path / "SCORE.MLG"
score_succeeded_text = "Duration:"
if analysisStatusChecker.checkIfAnalysisOk(score_succeeded_text, str(logFileToCheck)):
    print("Precompute run succeeded")
else:
    raise Warning|(
        "Precompute run failed - please check the log file for more information: "
        + str(logFileToCheck)
    )

precompute_output_files = output_folder_path.glob("*.sin")
if not precompute_output_files:
    raise FileNotFoundError("No precompute .sin files were found.")

print("The following .sin files were found:")
sin_count = 0
for file in precompute_output_files:
    print(file)
    sin_count += 1

# Rudimentary sanity check - there should be 8 sin files
if sin_count < 8:
    raise Warning(
        "Not all .sin files were generated! Please check the log file for more information: "
        + logFileToCheck
    )


Verifying pre-compute command result:
Precompute run succeeded
The following .sin files were found:
c:\source\sesam-time-domain-examples-internal\response-reconstruction\EMULF_FLS_Example_RR\Workspace\Precompute\Output\DiR3.sin
c:\source\sesam-time-domain-examples-internal\response-reconstruction\EMULF_FLS_Example_RR\Workspace\Precompute\Output\RaRxR3.sin
c:\source\sesam-time-domain-examples-internal\response-reconstruction\EMULF_FLS_Example_RR\Workspace\Precompute\Output\RaRyR3.sin
c:\source\sesam-time-domain-examples-internal\response-reconstruction\EMULF_FLS_Example_RR\Workspace\Precompute\Output\RaRzR3.sin
c:\source\sesam-time-domain-examples-internal\response-reconstruction\EMULF_FLS_Example_RR\Workspace\Precompute\Output\RaTxR3.sin
c:\source\sesam-time-domain-examples-internal\response-reconstruction\EMULF_FLS_Example_RR\Workspace\Precompute\Output\RaTyR3.sin
c:\source\sesam-time-domain-examples-internal\response-reconstruction\EMULF_FLS_Example_RR\Workspace\Precompute\Output\RaT

## 3. Response Reconstruction
1. Read parameters from Excel spreadsheet
2. Generate necessary input files
3. Run response reconstruction in Sesam Core for each DLC

The commands for input file generation and running Sesam Core are chained together into a workflow that is executed with OneWorkflow. OneWorkflow will take care of copying the necessary input files into the execution folders, and orchestrate the workflow

In [4]:
import pandas as pd
from dnv.oneworkflow.utils import (
    one_workflow_client,
    run_managed_commands_in_parallel_async
)
parameter_input_file = workspace_path / excel_file
parameters_from_excel = pd.read_excel(parameter_input_file, index_col=0)

# Create a OneWorkflow client
workflow_client = one_workflow_client(
    workspace_id = workspace_id,
    workspace_path = str(workspace_path),
    cloud_run = cloud_run,
    workspace_common_folder_name = str(common_files_folder),
    inplace_execution = True
)

local_result_path = workspace_path / workflow_client.results_directory
lc_folder = workspace_path / workflow_client.load_cases_directory

parameter_mapping = {
    "StartTime": "start_solve",
    "EndTime": "stop_solve",
    "TimeStep": "timestep",
    "Nsteps": "nsteps",
}
run_name = "screeningrun1"
expected_output_files = []
workflow_sequence = []

for lc, case in parameters_from_excel.iterrows():
    print("Processing load case: " + lc)

    tasks = []

    this_lc_path = lc_folder / lc

    if this_lc_path.exists() and run_reconstruction is True:
        print("Load case folder already exists, deleting it")
        shutil.rmtree(this_lc_path, ignore_errors=True)
        this_lc_path.mkdir(parents=True, exist_ok=True)

    expected_output_files.append(Path(this_lc_path, f"{run_name}F{top_sup}.SIN"))
    expected_output_files.append(Path(this_lc_path, f"SesamCore_{run_name}.lis"))
    casedict = case.to_dict()

    if run_reconstruction:
        input_parameters = {}

        # find the values from the Excel sheet for give load case
        for key, value in case.items():
            input_parameters[parameter_mapping[key]] = str(value)

        input_parameters["FATIGUESTART"] = float(input_parameters["start_solve"])
        input_parameters["FATIGUEEND"] = float(input_parameters["stop_solve"])
        input_parameters["fem"] = f"{fem_file_prefix}T{top_sup}.FEM"
        input_parameters["runname"] = run_name

        tasks.append(
            CreateInputFileFromFileTemplateCommand(
                template_input_file="SesamCore_screening_template.jnl",
                input_filename="SesamCore_screening.jnl",
                parameters=input_parameters,
            )
        )

        tasks.append(
            CreateInputFileFromFileTemplateCommand(
                template_input_file="input_template.json",
                input_filename="input.json",
                parameters=input_parameters,
            )
        )

        tasks.append(
            SesamCoreCommand(
                working_dir=str(workspace_path),
                command="fatigue",
                input_file_name="input.json",
                options="-rec -s",
            )
        )

        # Add the generated tasks to the workflow sequence.
        workflow_sequence.append(
            CommandInfo(commands=tasks, load_case_foldername=lc)
        )

if run_reconstruction:
    print("Running Response Reconstruction in parallel")
    await run_managed_commands_in_parallel_async(
        client=workflow_client,
        commands_info=workflow_sequence,
        log_job=False,
        files_to_exclude_from_common_files_copy=["*.sin"],
        enable_common_files_copy_to_load_cases=True,
    )
    print("Finished running Response Reconstruction")


Processing load case: LC001
Processing load case: LC002
Running Response Reconstruction in parallel
Info: Attempt 1/10.LocalWorkflowRuntime service (PID '31280') is not ready yet. Retrying in 5 seconds.
Info: The LocalWorkflowRuntime service (PID '31280') is ready.
Info: The work item 5e57ce50-69e8-4920-a6da-f8d52e606f26 message is 'Created'
Info: The work item LC001 message is 'Created'
Info: The work item LC002 message is 'Created'
Info: The progress of the job is '0%'. The message is ''
Info: The work item LC001 message is ''
Info: The work item LC002 message is ''
Info: The work item LC001 message is 'Executing SesamCore (score.exe).'
Info: The work item LC002 message is 'Executing SesamCore (score.exe).'
Info: The progress of the job is '75%'. The message is 'Completed 0 of 2 tasks of task group 5e57ce50-69e8-4920-a6da-f8d52e606f26'
Info: The status of work item '5e57ce50-69e8-4920-a6da-f8d52e606f26' is 'Completed'
Info: The status of work item 'LC001' is 'Completed'
Info: The sta

Verify that response reconstruction run was successful

In [5]:
import os
cwd = os.getcwd()
try:
    os.chdir(lc_folder)
    if not analysisStatusChecker.checkStatus(
        {"Sesam Core": "SCORE.MLG"},
        "Reaction forces summed over degrees of freedom with boundary boundary constraints",
    ) or not analysisStatusChecker.checkExpectedFiles(expected_output_files):
        raise Warning("Analysis failed, please see any relevant log files (1)")
except:
    print("Analysis failed, please see any relevant log files")
finally:
    os.chdir(cwd)


Assuming results to be in folder  c:\source\sesam-time-domain-examples-internal\response-reconstruction\EMULF_FLS_Example_RR\Workspace\LoadCases
Analysis was successful for all loadcases for program Sesam Core.


## 4. FLS result accumulation

Some user settable parameters for the FLS result accumulation

In [6]:
### USER INPUT ###

# ElementScreening, HotSpotPlate or HotSpotMultiDirectional
fls_check_type = "ElementScreening"

# AbsoluteOccurrence, RelativeOccurrence or Probability
probability_mode = "AbsoluteOccurrence"

# Accumulation period in years (not needed for AbsoluteOccurrence mode)
accumulation_in_years = 20

# Use only selected elements for calculating occurrence factor of load cases.
use_selected_only = False

# Name of the screening capacity model
screening_capacity_model = "fat_screen.json"

### END USER INPUT ###

First generate a template .csv file

In [7]:
# Generate template file
template_file = workspace_path / "template.csv"
if template_file.exists():
    template_file.unlink()

score_accumulate_templat = SesamCoreCommand(working_dir = str(workspace_path))
score_accumulate_templat.arguments = f"accumulation generate-load-cases-template -o {str(template_file)}"

execute_command(score_accumulate_templat)

load_cases_data = pd.read_csv(template_file)

Fill the template file with information read from the Excel file

In [8]:
for lc, case in parameters_from_excel.iterrows():
    this_lc_path = Path(lc_folder, lc)
    sin_files = this_lc_path.glob("*.SIN")

    input_parameters = {}

    # find the values from the Excel sheet for given load case
    for key, value in case.items():
        input_parameters[parameter_mapping[key]] = str(value)

    for sin_file in sin_files:
        print(f"Found SIN file: {sin_file.relative_to(workspace_path)}")
        df = pd.DataFrame(
            {
                "IsSelected": [True],  # default value
                "Name": [lc],
                "StartTime": [input_parameters["start_solve"]],
                "EndTime": [input_parameters["stop_solve"]],
                "SimulationLength": [float(input_parameters["stop_solve"]) - float(input_parameters["start_solve"])],
                "NumberOfOccurrences": [1],  # default value
                "ResultsFilePath": [str(sin_file)],
            }
        )
        with open(template_file, "ab") as f:
            df.to_csv(f, header=False, index=False, sep=";")


Found SIN file: LoadCases\LC001\screeningrun1F3.SIN
Found SIN file: LoadCases\LC002\screeningrun1F3.SIN


#### Run the FLS result accumulation

In [15]:
import glob
screening_capacity_model_path = input_folder_path / screening_capacity_model
accumulation_dir = workspace_path / "AccumulatedResults"
accumulation_input_file_path = accumulation_dir / "accumulation-input.json"
if run_accumulation:
    # remove accumulation folder if it exists
    if accumulation_dir.exists():
        shutil.rmtree(accumulation_dir, ignore_errors=True)
    accumulation_dir.mkdir()

    score_args = f"accumulation generate-input \
                -t {fls_check_type} \
                -m {probability_mode} \
                -cm {screening_capacity_model_path} \
                -lcd {template_file} \
                -at {accumulation_in_years} \
                -so:{use_selected_only} \
                -o {accumulation_input_file_path}"

    score_generate_input = SesamCoreCommand(working_dir=str(accumulation_dir))
    score_generate_input.arguments = score_args

    print("Running accumulation generate-input command")
    execute_command(score_generate_input)

    score_accumulate = SesamCoreCommand(working_dir=str(accumulation_dir))
    score_accumulate.arguments = (
        f"accumulation accumulate -i {accumulation_input_file_path} -co"
    )

    print("Running accumulation command")
    execute_command(score_accumulate)
    print("Finished running accumulation command")
    if os.path.exists(accumulation_dir):
        print("Accumulated results can be found in: " + str(accumulation_dir)+ ":")
        for file in os.listdir(accumulation_dir):
            print(accumulation_dir/file)
        #the result file has a time stamp so we need to find the correct file
        result_files = glob.glob(os.path.join(accumulation_dir,"accumulation-results-ElementScreening*.json"))
        if len(result_files) > 0:
            result_file = result_files[0]
            #print("Accumulated results are stored in: " + str(accumulation_dir / result_file[0]))
            df = pd.read_json(accumulation_dir / result_file)
    else:
        print("Accumulated results were not found in: " + str(accumulation_dir) )
    


Running accumulation generate-input command
Running accumulation command
Finished running accumulation command
Accumulated results can be found in: c:\source\sesam-time-domain-examples-internal\response-reconstruction\EMULF_FLS_Example_RR\Workspace\AccumulatedResults:
c:\source\sesam-time-domain-examples-internal\response-reconstruction\EMULF_FLS_Example_RR\Workspace\AccumulatedResults\accumulation-input.json
c:\source\sesam-time-domain-examples-internal\response-reconstruction\EMULF_FLS_Example_RR\Workspace\AccumulatedResults\accumulation-results-ElementScreening-2024-07-12-12-55-15.json
c:\source\sesam-time-domain-examples-internal\response-reconstruction\EMULF_FLS_Example_RR\Workspace\AccumulatedResults\SCORE.MLG


#### Display a table of the accumulated results:

In [31]:
import json
from IPython.display import HTML
screening_file = workspace_path /common_files_folder / "fat_screen.json"
checkpoint_to_element_map = {}

# first create a mapping between CheckPointId and ElementId using the original screening file|
with open(screening_file,'r') as file:
    screening_data = json.load(file)
for element in screening_data["ScreeningElements"]:
    # Extract CheckPointId and Element
    checkpoint_id = element["CheckPointId"]
    element_id = element["Element"]

    # Store in the dictionary
    checkpoint_to_element_map[checkpoint_id] = element_id
print(checkpoint_to_element_map)
results_table =[]

# read in the screening results and extract the relevant information
result_files = glob.glob(os.path.join(accumulation_dir,"accumulation-results-ElementScreening*.json"))
print(result_files)
if len(result_files) > 0:
    result_file = result_files[0]
    with open(result_file, 'r') as file:
        json_data = json.load(file)
# Process the JSON data
for item in json_data:
    checkpoint_id = item["CheckPoint"]["id"]
    plate_side = item["Fatigue"]["plateSide"]
    damage = item["Fatigue"]["damage"]
    
    # Use the checkpoint_to_element_map to find the corresponding Element Number
    element_number = checkpoint_to_element_map.get(checkpoint_id)
    
    # Append the extracted information as a dictionary to the rows list
    results_table.append({
        "Checkpoint ID": checkpoint_id,
        "Element Number": element_number,
        "Plate Side": plate_side,
        "Damage": damage
    })

# Convert the list of dictionaries into a pandas DataFrame
df = pd.DataFrame(results_table)
df.to_string(index=False)
# Adjust the display settings to show all rows
pd.options.display.max_rows = 800
display(HTML(df.to_html(index=False)))


{1: 9583, 2: 9584, 3: 9585, 4: 9586, 5: 9587, 6: 9588, 7: 9589, 8: 9590, 9: 9591, 10: 9592, 11: 9593, 12: 9594, 13: 9595, 14: 9596, 15: 9597, 16: 9598, 17: 9599, 18: 9600, 19: 9601, 20: 9602, 21: 9603, 22: 9604, 23: 9605, 24: 9606, 25: 9607, 26: 9608, 27: 9609, 28: 9610, 29: 9611, 30: 9612, 31: 9613, 32: 9614, 33: 9615, 34: 9616, 35: 9617, 36: 9618, 37: 9619, 38: 9620, 39: 9621, 40: 9622, 41: 9623, 42: 9624, 43: 9625, 44: 9626, 45: 9627, 46: 9628, 47: 9629, 48: 9630, 49: 9631, 50: 9632, 51: 9633, 52: 9634, 53: 9635, 54: 9636, 55: 9637, 56: 9638, 57: 9639, 58: 9640, 59: 9641, 60: 9642, 61: 9643, 62: 9644, 63: 9645, 64: 9646, 65: 10572, 66: 10573, 67: 10574, 68: 10575, 69: 10576, 70: 10577, 71: 10578, 72: 10579, 73: 10580, 74: 10581, 75: 10582, 76: 10583, 77: 10584, 78: 10585, 79: 10586, 80: 10587, 81: 10588, 82: 10589, 83: 10590, 84: 10591, 85: 10592, 86: 10593, 87: 10594, 88: 10595, 89: 10596, 90: 10597, 91: 10598, 92: 10599, 93: 10600, 94: 10601, 95: 10602, 96: 10603, 97: 10604, 98: 1

Checkpoint ID,Element Number,Plate Side,Damage
412,13580,-z-side,3.156321e-02
4300,39608,-z-side,1.136711e-02
4337,39836,-z-side,9.982983e-03
4209,39066,-z-side,9.588388e-03
412,13580,+z-side,8.769477e-03
4390,40315,+z-side,5.895046e-03
4357,40005,-z-side,4.981895e-03
413,13581,+z-side,4.079579e-03
4306,39630,+z-side,3.701239e-03
4072,38532,+z-side,3.459686e-03
